## Get the company tickers

For reference, we can obtain the company tickers for the publicly traded funds.  SEC maintains a list in a `json` file.  Tickers can be useful for Google searches; they are unique, and are much more common than the SEC CIK identifier.

In [ ]:
import numpy as np
import pandas as pd
import requests
import json

headers = {'User-Agent':'robot.games@gmail.com'}
temp = requests.get("https://www.sec.gov/files/company_tickers.json",headers=headers).json()
company_tickers = pd.DataFrame(temp.values()).rename({'cik_str':'cik'})

In [ ]:
print(company_tickers.shape)
company_tickers.head()

# Get the Filer's SEC Info

See https://www.sec.gov/open/datasets-investment_company for information.

In [ ]:
import pandas as pd
info = pd.read_csv("data/investment_company_series_class_2020.csv",dtype = {"CIK Number":str})

In [ ]:
info.head()

We want to grab only certain types of entity, as classified by the `Entity Org Type` column.  How are the values there distributed?

In [ ]:
info['Entity Org Type'].value_counts()

In [ ]:
import seaborn as sns
sns.barplot(info['Entity Org Type'].value_counts())

We only want type 30.  Let's get it.

In [ ]:
info = info[info["Entity Org Type"]==30]

In [ ]:
info.head()

Let's get and store the allowed CIK numbers.

In [ ]:
allowed_filers = (
    info[["CIK Number",'Entity Name','Series ID','Series Name']]
    .rename(columns = {'CIK Number':'cik','Entity Name':'name','Series ID':'series_id','Series Name':'series_name'})
)
allowed_filers.head()

We can join data to this later and remove NA rows to filter.

## Get the Edgar API information

The API information allows us to search for the keys that let us open the NPORT documents.  So...
1. Get API information.
2. Extract two numbers that uniquely define the online location of the NPORT form.
3. Go get that NPORT form.

In [ ]:
from selenium import webdriver

url = 'https://www.sec.gov/Archives/edgar/full-index/2020/QTR1/form.zip'
download_directory = 'C:\\Users\\robot\\Downloads\\'

driver = webdriver.Firefox()
driver.implicitly_wait(20)
# driver.get(url)  # selenium is hanging on this.  Want to fgure this out but have manually downloaded the needed file and stored locally.
driver.close()

Move the zipfile into the data directory of the project folder.  Then extract it.

In [ ]:
import zipfile
with zipfile.ZipFile('data/form.zip') as zip_object:
    zip_object.extractall('data')

Open the index file.

In [ ]:
import pandas as pd
with open('data/form.idx','r') as f:
    temp = pd.DataFrame({'line':f.readlines()})
temp = temp[10:]

Here's what the first line looks like.  Essentially we need to grab the two numbers in the file name, reformat them and that lets us build the name of the file online.  But we only want the information from the NPORT filings; the first line here is from a 1-A filing and so we will not use that.

In [ ]:
list(temp.line)[0]

Now grab the URLs for the NPORT forms wherever mentioned in the index file.  There is some `regex` (regular expressions) magic here.

In [ ]:
# use only the NPORT filings
nport_lines = temp[ temp.line.str.find('NPORT-')>=0 ]
# get the needed data
nport_split = nport_lines.line.str.replace('[ \t\n]{2,}',' ',regex=True).str.split(' ')
# reformat to part of a URL
nport_url = [nport_split.iloc[z][-2] for z in range(nport_split.shape[0])]

For example,

In [ ]:
nport_url[0]

## Locate and download each NPORT form

I performed these operations once and stored the files locally.  No need to repeat, but the code can be easily adapted to another fiancial quarter.

In [ ]:
url_stem = 'https://www.sec.gov/Archives/'
nport_txt_doc_url = [url_stem + x for x in nport_url]

In [ ]:
nport_txt_doc_url[0]

Get all of the xml files and store them locally.

In [ ]:
import requests
import time
N = len(nport_txt_doc_url)
s = requests.Session()
for i in range(N):
    try:
        #print('Working on file '+str(i)+'/'+str(N))
        headers = {'User-Agent':'robot.games@gmail.com'}
        temp = s.get(nport_txt_doc_url[i],headers=headers,timeout=20)
    except:
        print('Timeout on item '+str(i))
    time.sleep(0.1)
    with open('data/xml/'+str(i)+'.txt','wb') as f:
        f.write(temp.content)

# Parsing the XML Documents

Parse each xml document (each NPORT form) one by one.  Gather the results into two data frames and store them on disk as csv files.

Total execution time: ~ 7 hours.

Set up the initial data buckets.  Dictionaries are easy to work with here and are easy to convert to data frames.

In [ ]:
filer = {
    'name':[],
    'cik':[],
    'series':[],
    'series_id':[],
    'total_assets':[],
    'total_liabilities':[],
    'cash':[],
    'file_name':[]
}

investment = {
    'cik':[],
    'series_id':[],
    'investment_name':[],
    'isin':[],
    'value_USD':[],
    'percentage_investment':[],
    'asset_category':[],
    'issuer_category':[],
    'fair_value_level':[],
    'file_name':[]
}

Define a few helper functions to perform the different steps in the extraction process.

In [ ]:
from bs4 import BeautifulSoup
import pandas as pd
import os

def get_filer_information(soup):
    regname = soup.select('regName')
    if len(regname)>0:
        filer_name = regname[0].text.replace(',','')
    else:
        print('NPORT skipped')
        return
    cik_value = soup.select('regCik')[0].text
    series = soup.select('seriesName')[0].text
    series_id = soup.select('seriesId')
    if len(series_id)>0:
        series_id = series_id[0].text
    else:
        series_id = None
    assets = float(soup.select('totAssets')[0].text)
    liabilities = float(soup.select('totLiabs')[0].text)
    try:
        cash = float(soup.select('cshNotRptdInCorD')[0].text)
    except:
        cash = 0.0
    return (filer_name,cik_value,series,series_id,assets,liabilities,cash)

def get_investment_information(node):
    try:
        isnode = node
        while (isnode.name != 'identifiers') and not (isnode is None):
            isnode = isnode.next_sibling
        if not(isnode is None):
            isnode = next(isnode.children,None)
            while (isnode.name != 'isin') and not (isnode is None):
                isnode = isnode.next_sibling
        vnode = node
        while (vnode.name != 'valusd') and not(vnode is None):
            vnode = vnode.next_sibling
        if not(vnode is None):
            pnode = vnode
        else:
            pnode = node
        while (pnode.name != 'pctval') and not(pnode is None):
            pnode = pnode.next_sibling
        if not(pnode is None):
            anode = pnode
        else:
            anode = node
        while (anode.name != 'assetcat') and not(anode is None):
            anode = anode.next_sibling
        if not(anode is None):
            inode = anode
        else:
            inode = node
        while (inode.name != 'issuercat') and not(inode is None):
            inode = inode.next_sibling
        if not(inode is None):
            fnode = inode
        else:
            fnode = node
        while (fnode.name != 'fairvallevel') and not(fnode is None):
            fnode = fnode.next_sibling
        isin = isnode.get('value')
        value_USD = float(vnode.text)
        percent = float(pnode.text)
        asset_type = anode.text
        issuer_type = inode.text    
        fair_value_level = fnode.text
    except:
        return
    return (isin,value_USD, percent,asset_type,issuer_type,fair_value_level)

def parse_xbrl_file(handle):
    # Throw everything into beautifulsoup
    soup = BeautifulSoup(handle,'lxml')
    filer_information = get_filer_information(soup)
    # This is tricky because there are some missing values in some files
    investment_names = soup.select('name')
    print('There are',len(investment_names),'investments to parse')
    investment_information = {}
    for node in investment_names:
         investment_information[node.text] = get_investment_information(node)
    return (filer_information,investment_information)

Perform the operation.  This could possibly be ported to parallel processing and cut the time down by a far bit.  However, I'm only performing these operations once or twice for this research project; if we were performing this operation four times a year I would implement this.

In [ ]:
file_names = ['data/xml/'+z for z in os.listdir('data/xml')]
start = 9000
end = len(file_names)
file_names = file_names[start:end]
for i,file_name in enumerate(file_names):
        print('Working on',file_name,'which is',i,'/',end-start)
        with open(file_name,'r') as handle:
                #  Add to the filer database
                filer_information,investment_information = parse_xbrl_file(handle)
                try:
                        filer_name,cik_value,series,series_id,assets,liabilities,cash = filer_information
                        filer['name'].append(filer_name)
                        filer['cik'].append(cik_value)
                        filer['series'].append(series)
                        filer['series_id'].append(series_id)
                        filer['total_assets'].append(assets)
                        filer['total_liabilities'].append(liabilities)
                        filer['cash'].append(cash)
                        filer['file_name'].append(file_name)
                except:
                        print('skipping one filer')
                for i_name in list(investment_information.keys()):
                        print(i_name)
                        if not(investment_information[i_name] is None):
                                isin,value_USD,percent,asset_type,issuer_type,fair_value_level = investment_information[i_name]
                                investment['isin'].append(isin)
                                investment['cik'].append(cik_value)
                                investment['series_id'].append(series_id)
                                investment['investment_name'].append(i_name)
                                investment['value_USD'].append(value_USD)
                                investment['percentage_investment'].append(percent)
                                investment['asset_category'].append(asset_type)
                                investment['issuer_category'].append(issuer_type)
                                investment['fair_value_level'].append(fair_value_level)
                                investment['file_name'].append(file_name)


In [ ]:
filer = pd.DataFrame(filer)
investment = pd.DataFrame(investment)
filer.replace(',','',regex=True,inplace=True)
investment.replace(',','',regex=True,inplace=True)

In [ ]:
investment.head()

Write out the initial files to local storage.

IF WRITING NEW:

In [ ]:
filer.to_csv('filer_raw.csv',index=False)
investment.to_csv('investment_raw.csv',index=False)

IF APPENDING:

In [ ]:
import pandas as pd
filer_disk = pd.read_csv('filer_raw.csv',dtype={'cik':str})
investment_disk = pd.read_csv('investment_raw.csv',dtype={'cik':str})
filer2 = pd.concat([filer_disk,filer],ignore_index=True)
investment2 = pd.concat([investment_disk,investment],ignore_index=True)
filer2.to_csv('filer_raw.csv',index=False)
investment2.to_csv('investment_raw.csv',index=False)

In [ ]:
filer2.shape

In [ ]:
investment2.shape

## Explore, filter, and validate the data

Load the raw csv files.

In [ ]:
import pandas as pd
filer = pd.read_csv('filer_raw.csv',dtype={'cik':str})
investment = pd.read_csv('investment_raw.csv',dtype={'cik':str})

The `filer` data contains data about the entity filing the NPORT form.

In [ ]:
filer.head()

Let's filter that for the allowed cik and series values.

In [ ]:
filer = (
    allowed_filers
    [['cik','series_id']]
    .merge(filer,how='right',on=['cik','series_id'])
    .dropna()
    .drop_duplicates()
)
investment = (
    allowed_filers
    [['cik','series_id']]
    .merge(investment,how='right',on=['cik','series_id'])
    .dropna()
    .drop_duplicates()
)

In [ ]:
investment.head()

In [ ]:
investment_summary = (
    investment
    [['cik','series_id','value_USD','percentage_investment']]
    .groupby(['cik','series_id'])
    .sum()
    .rename(columns={'value_USD':'total_value_USD','percentage_investment':'total_percentage_invested'})
    .reset_index()
)

In [ ]:
investment = (
    investment
    .merge(investment_summary,how='left',on=['cik','series_id'])
)
investment['fraction_of_value'] = investment.value_USD / investment.total_value_USD
investment['fraction_of_stated_percentage'] = investment.percentage_investment / investment.total_percentage_invested

In [ ]:
investment.head()

In [ ]:
filer.to_csv('filer_raw2.csv',index=False)
investment.to_csv('investment_raw2.csv',index=False)

## Chunking the file for upload to github

In [ ]:
import pandas as pd
def write_chunk_csv(df,n,file_stem='out'):
    N = round(df.shape[0]/n)
    i=0
    while i < n-1:
        temp = df.iloc[(i*N):((i+1)*N),:]
        temp.to_csv(file_stem+'_'+str(i)+'.csv',index=False)
        i+=1
    temp = df.iloc[((n-1)*N):,:]
    temp.to_csv(file_stem+'_'+str(n-1)+'.csv',index=False)

In [ ]:
write_chunk_csv(investment,20,file_stem='investment')